# T23_G8 - 01 - Import

## Setup

In [454]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

pd.set_option('display.max.columns', None)

import pyreadstat

## Reading Codebook

* I just want to make sure that I understand the relationship between the codebook and the datafile.

In [455]:
# Read in codebook (all worksheets - first three letters (in lowercase) give name of data file.)
xls = pd.ExcelFile("orig/docs/T23_G8_Codebook.xlsx")
xls.sheet_names

['BCGM8', 'BSAM8', 'BSGM8', 'BSPM8', 'BSRM8', 'BSTM8', 'BTMM8', 'BTSM8']

In [456]:
# Compiled from User Guide T23 User Guide, (exhibit 2.4, p.52)
DATAFILES = {
    'bcg': 'School context data',
    'bsa': 'Student achievement data',
    'bsp': 'Student process data',
    'bsr': 'Within-country scoring reliability data',
    'bsg': 'Student context data data',
    'bst': 'Student achievement data',
    'btm': 'Mathematics teacher context data',
    'bts': 'Science teacher context data (we will ignore this)',
}

### Check: codebook matches datafiles

Iterating over all worksheets:

* Verify that variables (columns) in data file matches variables (rows) in codebook worksheet.
* Print size of datafile (I was curious).

In [457]:
for work_sheet in xls.sheet_names:

    datafile = work_sheet[:3].lower()
    print(f"Datafile: {datafile} : {DATAFILES[datafile]} ...")

    # Load codebook and datafile
    df_codebook = pd.read_excel(xls, sheet_name=work_sheet)
    df_data = pd.read_spss(f"orig/SPSS/{datafile}irlm8.sav")
 
    # Extract variable names from codebook and datafile
    variables_in_codebook = df_codebook.Variable.values.tolist()
    variables_in_datafile = df_data.columns.tolist()

    # Sneaky peak into datafile    
    print(f"\t Shape:   {df_data.shape}")
    print(f"\t Columns: {df_data.columns.tolist()[:10]}")

    # Demand variables in codebook and in datafile match
    assert variables_in_codebook==variables_in_datafile, "Mismatch between codebook and datafile for data file type {datafile}"

Datafile: bcg : School context data ...
	 Shape:   (153, 104)
	 Columns: ['CTY', 'IDCNTRY', 'IDPOP', 'IDGRADER', 'IDGRADE', 'IDSCHOOL', 'ITLANG_CQ', 'LCID_CQ', 'BCBG03A', 'BCBG03B']
Datafile: bsa : Student achievement data ...


	 Shape:   (5090, 1145)
	 Columns: ['CTY', 'IDCNTRY', 'IDPOP', 'IDGRADER', 'IDGRADE', 'ITASSESS', 'IDBOOK', 'IDSCHOOL', 'IDCLASS', 'IDSTUD']
Datafile: bsg : Student context data data ...
	 Shape:   (5090, 511)
	 Columns: ['CTY', 'IDCNTRY', 'IDPOP', 'IDGRADER', 'IDGRADE', 'ITASSESS', 'IDBOOK', 'IDSCHOOL', 'IDCLASS', 'IDSTUD']
Datafile: bsp : Student process data ...
	 Shape:   (5090, 1304)
	 Columns: ['CTY', 'IDCNTRY', 'IDPOP', 'IDGRADER', 'IDGRADE', 'ITASSESS', 'IDBOOK', 'IDSCHOOL', 'IDCLASS', 'IDSTUD']
Datafile: bsr : Within-country scoring reliability data ...
	 Shape:   (4723, 662)
	 Columns: ['CTY', 'IDCNTRY', 'IDPOP', 'IDGRADER', 'IDGRADE', 'IDBOOK', 'IDSCHOOL', 'IDCLASS', 'IDSTUD', 'MER72002']
Datafile: bst : Student achievement data ...
	 Shape:   (9992, 122)
	 Columns: ['CTY', 'IDCNTRY', 'IDPOP', 'IDGRADER', 'IDGRADE', 'ITASSESS', 'IDBOOK', 'IDSCHOOL', 'IDCLASS', 'IDSTUD']
Datafile: btm : Mathematics teacher context data ...
	 Shape:   (630, 169)
	 Columns: ['CTY', 'IDCNTRY', '

So some columns are repeated across datafiles. Of course this should happen &mdash; how else could the datafile be joined?

So on possible step would be to take all eight worksheets and join them into one dataframe and dropping duplicates. This data frame will be used whenever I need to lookup meta information about a variable.

In [458]:
# Read codebook in (again, just to be sure) and concat all sheets into one dataframe

xls = pd.ExcelFile("orig/docs/T23_G8_Codebook.xlsx")
dfs = [pd.read_excel(xls, sheet_name=s) for s in xls.sheet_names]
df_codebook = pd.concat(dfs)

print(df_codebook.shape)
df_codebook.head(1)

(4235, 16)


,Variable,Label,Question Location,Level,Width,Decimals,Range Minimum,Range Maximum,Value Scheme Detailed,Missing Scheme Detailed: SPSS,Missing Scheme Detailed: SAS,Field Code: SPSS,Field Code: SAS,Domain,Variable Class,Comment
0,CTY,Country Alpha3,NaN,Not defined,5,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Calculated at IEA,NaN,NaN


### Check: Variables have single purpose

Some variables appear in multiple datasets. They SHOULD have same meaning in each dataset.

If I drop duplicates, then each variable should appear exactly once. 

In [459]:
print(df_codebook.shape)
df_codebook = df_codebook.drop_duplicates()
print(df_codebook.shape)

(4235, 16)
(3543, 16)


The following assertion should pass,

```python
assert df_codebook['Variable'].value_counts().max()==1, "Some variable has differing entries"
``` 

but someone (not me, this time) has made a mistake in the data and out of the 3542 variables in the dataset one variable (`ITLANG_SA`) appears twice.

In [460]:
df_tmp = df_codebook['Variable'].value_counts()

# Do any variables have multiple (different) meta information? (should be none)
df_tmp[df_tmp>1]

Variable
ITLANG_SA    2
Name: count, dtype: int64

In [461]:
df_codebook.query("Variable=='ITLANG_SA'")

,Variable,Label,Question Location,Level,Width,Decimals,Range Minimum,Range Maximum,Value Scheme Detailed,Missing Scheme Detailed: SPSS,Missing Scheme Detailed: SAS,Field Code: SPSS,Field Code: SAS,Domain,Variable Class,Comment
14,ITLANG_SA,Language of Student Achievement Test,NaN,Nominal,2,0,NaN,NaN,1: English; 2: Spanish; 3: French; 4: Afrikaan...,99: Omitted or invalid; Sysmis: Not administered,.: Omitted or invalid; .A: Not administered,1: English; 2: Spanish; 3: French; 4: Afrikaan...,1: English; 2: Spanish; 3: French; 4: Afrikaan...,Calculated at IEA,IEA_D,NaN
14,ITLANG_SA,Language of Student Achievement Test,NaN,Nominal,2,0,NaN,NaN,1: English; 2: Spanish; 3: French; 4: Afrikaan...,99: Omitted or invalid; Sysmis: Not administered,.: Omitted or invalid; .A: Not administered,1: English; 2: Spanish; 3: French; 4: Afrikaan...,1: English; 2: Spanish; 3: French; 4: Afrikaan...,Calculated at IEA,IEA_D,NaN


Fortunately, this variable is unlikely to be of importance to us, looking at the Ireland data, so I'm going to delete one of the rows.

In [462]:
print(df_codebook.shape)
df_codebook = df_codebook.drop_duplicates(subset=['Variable'])
print(df_codebook.shape)

(3543, 16)
(3542, 16)


Now the assertion works as expected

In [463]:
assert df_codebook['Variable'].value_counts().max()==1, "Some variable has differing entries"

Finally I set the index of the codebook to the variable name.

In [464]:
if not df_codebook.index.name:
    df_codebook = df_codebook.set_index('Variable')

df_codebook.head(1)

,Label,Question Location,Level,Width,Decimals,Range Minimum,Range Maximum,Value Scheme Detailed,Missing Scheme Detailed: SPSS,Missing Scheme Detailed: SAS,Field Code: SPSS,Field Code: SAS,Domain,Variable Class,Comment
Variable,,,,,,,,,,,,,,,
CTY,Country Alpha3,NaN,Not defined,5,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Calculated at IEA,NaN,NaN


### Sample usage of codebook

In [465]:
# get everything about a variable
df_codebook.loc['ITLANG_SA']

Label                                         Language of Student Achievement Test
Question Location                                                              NaN
Level                                                                      Nominal
Width                                                                            2
Decimals                                                                         0
Range Minimum                                                                  NaN
Range Maximum                                                                  NaN
Value Scheme Detailed            1: English; 2: Spanish; 3: French; 4: Afrikaan...
Missing Scheme Detailed: SPSS     99: Omitted or invalid; Sysmis: Not administered
Missing Scheme Detailed: SAS           .: Omitted or invalid; .A: Not administered
Field Code: SPSS                 1: English; 2: Spanish; 3: French; 4: Afrikaan...
Field Code: SAS                  1: English; 2: Spanish; 3: French; 4: Afrikaan...
Doma

In [466]:
# get a single property about a variable
df_codebook.loc['ITLANG_SA']['Label']

'Language of Student Achievement Test'

In [467]:
# or could define a function (to separate implementation from use)
def get_property(variable, property='Label'):
    return  df_codebook.loc[variable][property]

get_property('IDGRADER')

'Standardized Grade ID'

## Read single datafile

Here I wanted to read in a single dataset to see what is the best approach to use.

In [468]:
datafile = 'bcg'
country = 'irl'
filename = f'{datafile}{country}m8.sav'
filename

'bcgirlm8.sav'

The SPSS `sav` file can be read using two methods:

* Using `pd.read_spss` will return a `pd.DataFrame` with ALL columns with `category` data type.
    * So numerical columns will need be converted. 
* Using `pyreadstat.read_sav` will return a `pd.DataFrame` and a `pyreadstat.metadata_container`.
    * The dataFrame is encoded (so label values are replaced by numbers) and all columns have `float` data type.
    * To use this we would have to undo the encoding to get understandable labels.


The following cells read in one datafile using both methods and compares the results.

### Using `pd.read_spss`

In [469]:
df_p = pd.read_spss(f'orig/SPSS/{filename}')
display(df_p.head(1))
df_p.dtypes

,CTY,IDCNTRY,IDPOP,IDGRADER,IDGRADE,IDSCHOOL,ITLANG_CQ,LCID_CQ,BCBG03A,BCBG03B,BCBG04,BCBG05A,BCBG05B,BCBG06A,BCBG06B,BCBG06C,BCBG07,BCBG08,BCBG09,BCBG10A,BCBG10B,BCBG11AA,BCBG11AB,BCBG11AC,BCBG11AD,BCBG11AE,BCBG11AF,BCBG11AG,BCBG11AH,BCBG11AI,BCBG11BA,BCBG11BB,BCBG11BC,BCBG11BD,BCBG11BE,BCBG11CA,BCBG11CB,BCBG11CC,BCBG11CD,BCBG11CE,BCBG12A,BCBG12B,BCBG12C,BCBG12D,BCBG12E,BCBG12F,BCBG12G,BCBG12H,BCBG12I,BCBG12J,BCBG12K,BCBG13A,BCBG13B,BCBG13C,BCBG13D,BCBG14A,BCBG14B,BCBG14C,BCBG14D,BCBG14E,BCBG14F,BCBG14G,BCBG14H,BCBG15A,BCBG15B,BCBG15C,BCBG15D,BCBG15E,BCBG15F,BCBG15G,BCBG15H,BCBG15I,BCBG15J,BCBG15K,BCBG16A,BCBG16B,BCBG17,BCBG18,BCBG19,BCBG20A,BCBG20B,BCBG20C,BCBG21A,BCBG21B,BCBG21C,BCBG21D,JKCZONE,JKCREP,SCHWGT,STOTWGTU,WGTFAC1,WGTADJ1,BCDGMRS,BCBGMRS,BCDGSRS,BCBGSRS,BCDGEAS,BCBGEAS,BCDGDAS,BCBGDAS,BCDGSBC,BCDGTIHY,VERSION,SCOPE
0,IRL,372.0,Pop 2,Upper Grade,Grade 8,5001.0,English,English (Ireland),More than 50%,0 to 10%,76 to 90%,"50,001 to 100,000 people",Medium size city or large town,166.0,NaN,4 1/2 days,90.0,Yes,Yes,Yes,Yes,A little,Not at all,A little,Not at all,A little,A little,A little,Some,Some,A little,A little,Some,Not at all,Some,Not at all,Not at all,A little,Not at all,A little,Very high,Very high,High,High,Low,Low,High,Medium,Medium,Medium,High,Agree a lot,Agree a little,Agree a little,Agree a lot,Agree a little,Agree a lot,Agree a little,Agree a lot,Agree a lot,Agree a lot,Agree a lot,Agree a lot,Minor problem,Minor problem,Minor problem,Not a problem,Minor problem,Not a problem,Minor problem,Minor problem,Minor problem,Not a problem,Not a problem,Minor problem,Not a problem,12.0,11.0,<Doctor or equivalent level—ISCED Level 8>,Yes,No,No,1 to 3 months,1 to 3 months,The school did not fully close,The school did not fully close,72.0,1.0,4.625,374.625,4.625,1.0,Somewhat Affected,9.95503,Somewhat Affected,10.63568,High Emphasis,9.83877,Minor Problems,10.10097,More Disadvantaged,NaN,4.0,Public Use File (PUF)


CTY           object
IDCNTRY     category
IDPOP       category
IDGRADER    category
IDGRADE     category
              ...   
BCBGDAS     category
BCDGSBC     category
BCDGTIHY    category
VERSION     category
SCOPE       category
Length: 104, dtype: object

### Using `pyreadstat.read_sav`

In [470]:
df_s, meta = pyreadstat.read_sav(f'orig/SPSS/{filename}')
type(df_s), type(meta)

(pandas.core.frame.DataFrame, pyreadstat._readstat_parser.metadata_container)

In [471]:
df_s.head()

,CTY,IDCNTRY,IDPOP,IDGRADER,IDGRADE,IDSCHOOL,ITLANG_CQ,LCID_CQ,BCBG03A,BCBG03B,BCBG04,BCBG05A,BCBG05B,BCBG06A,BCBG06B,BCBG06C,BCBG07,BCBG08,BCBG09,BCBG10A,BCBG10B,BCBG11AA,BCBG11AB,BCBG11AC,BCBG11AD,BCBG11AE,BCBG11AF,BCBG11AG,BCBG11AH,BCBG11AI,BCBG11BA,BCBG11BB,BCBG11BC,BCBG11BD,BCBG11BE,BCBG11CA,BCBG11CB,BCBG11CC,BCBG11CD,BCBG11CE,BCBG12A,BCBG12B,BCBG12C,BCBG12D,BCBG12E,BCBG12F,BCBG12G,BCBG12H,BCBG12I,BCBG12J,BCBG12K,BCBG13A,BCBG13B,BCBG13C,BCBG13D,BCBG14A,BCBG14B,BCBG14C,BCBG14D,BCBG14E,BCBG14F,BCBG14G,BCBG14H,BCBG15A,BCBG15B,BCBG15C,BCBG15D,BCBG15E,BCBG15F,BCBG15G,BCBG15H,BCBG15I,BCBG15J,BCBG15K,BCBG16A,BCBG16B,BCBG17,BCBG18,BCBG19,BCBG20A,BCBG20B,BCBG20C,BCBG21A,BCBG21B,BCBG21C,BCBG21D,JKCZONE,JKCREP,SCHWGT,STOTWGTU,WGTFAC1,WGTADJ1,BCDGMRS,BCBGMRS,BCDGSRS,BCBGSRS,BCDGEAS,BCBGEAS,BCDGDAS,BCBGDAS,BCDGSBC,BCDGTIHY,VERSION,SCOPE
0,IRL,372.0,2.0,2.0,8.0,5001.0,1.0,6153.0,4.0,1.0,2.0,3.0,3.0,166.0,NaN,4.0,90.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,2.0,2.0,2.0,3.0,3.0,2.0,2.0,3.0,1.0,3.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,2.0,2.0,4.0,4.0,2.0,3.0,3.0,3.0,2.0,1.0,2.0,2.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,1.0,2.0,2.0,2.0,1.0,1.0,2.0,1.0,12.0,11.0,4.0,1.0,2.0,2.0,3.0,3.0,1.0,1.0,72.0,1.0,4.625,374.625,4.625,1.0,2.0,9.95503,2.0,10.63568,2.0,9.83877,2.0,10.10097,3.0,NaN,4.0,1.0
1,IRL,372.0,2.0,2.0,8.0,5002.0,1.0,6153.0,3.0,2.0,3.0,5.0,3.0,168.0,360.0,3.0,176.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,2.0,3.0,3.0,1.0,1.0,72.0,0.0,4.625,351.500,4.625,1.0,1.0,12.35385,1.0,12.60240,1.0,16.29911,1.0,13.79710,3.0,1008.0,4.0,1.0
2,IRL,372.0,2.0,2.0,8.0,5003.0,1.0,6153.0,3.0,3.0,2.0,1.0,1.0,166.0,360.0,4.0,96.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,4.0,1.0,4.0,1.0,4.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,3.0,3.0,2.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,3.0,3.0,3.0,1.0,1.0,2.0,3.0,NaN,3.0,1.0,35.0,0.0,4.625,319.125,4.625,1.0,1.0,11.57754,1.0,12.13403,2.0,10.75751,2.0,10.10097,2.0,996.0,4.0,1.0
3,IRL,372.0,2.0,2.0,8.0,5004.0,1.0,6153.0,1.0,4.0,3.0,1.0,2.0,NaN,360.0,3.0,40.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,1.0,2.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.0,1.0,3.500,280.000,3.500,1.0,1.0,11.91488,1.0,11.47959,1.0,16.29911,1.0,12.65667,1.0,NaN,4.0,1.0
4,IRL,372.0,2.0,2.0,8.0,5005.0,1.0,6153.0,2.0,4.0,1.0,6.0,4.0,166.0,348.0,3.0,60.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,3.0,2.0,3.0,3.0,2.0,3.0,2.0,3.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,4.0,4.0,3.0,1.0,2.0,2.0,3.0,3.0,1.0,1.0,59.0,1.0,3.500,525.000,3.500,1.0,1.0,12.35385,1.0,12.13403,2.0,12.24516,2.0,10.42355,1.0,962.8,4.0,1.0


In [472]:
df_s.dtypes

CTY          object
IDCNTRY     float64
IDPOP       float64
IDGRADER    float64
IDGRADE     float64
             ...   
BCBGDAS     float64
BCDGSBC     float64
BCDGTIHY    float64
VERSION     float64
SCOPE       float64
Length: 104, dtype: object

The `metadata_container` stores meta information about the dataset and each column such as the type of data, the different names and labelling of columns, the encoding of values in the column. 

Use python `dir` to list public (do not start with '_') functions in `meta`. 

In [473]:
[s for s in dir(meta) if s[0]!='_']

['column_labels',
 'column_names',
 'column_names_to_labels',
 'creation_time',
 'file_encoding',
 'file_format',
 'file_label',
 'missing_ranges',
 'missing_user_values',
 'modification_time',
 'notes',
 'number_columns',
 'number_rows',
 'original_variable_types',
 'readstat_variable_types',
 'table_name',
 'value_labels',
 'variable_alignment',
 'variable_display_width',
 'variable_measure',
 'variable_storage_width',
 'variable_to_label',
 'variable_value_labels']

Some of the above functions might be useful. But my `get_property` function, defined above, covers nearly all my needs.

## Reversing label encoding 

The two dataframe, `df_p` and `df_s` store similar information. In `df_s` the data has been labeled encoded. Since the mapping for that encoding is stored in `meta` the encoding can be reversed.  

For example, if we take column `BCBG03A` we first perform `value_counts` to see that they store the same distribution, then we can decode the encoded values in `df_s` and verify that it is the same as the original data in `df_p`.  

This is just a curiosity &mdash; we can work with the original dataset, `df_p` without looking at `df_s` &mdash; to test the functionality of `pyreadstat`.

In [474]:
# show same distribution

column = 'BCBG03A' # pick a random column

label = meta.variable_to_label[column]
description = meta.column_names_to_labels[column]

print(f"Column: `{column}` with spss label `{label}`")
print(f"Description: `{description}`")

display(df_p[column].value_counts(dropna=False))
display(df_s[column].value_counts(dropna=False))

Column: `BCBG03A` with spss label `labels7`
Description: `GEN\STUDENTS BACKGROUND\ECONOMIC DISADVA`


BCBG03A
11 to 25%        50
0 to 10%         40
26 to 50%        24
More than 50%    23
NaN              16
Name: count, dtype: int64

BCBG03A
2.0    50
1.0    40
3.0    24
4.0    23
NaN    16
Name: count, dtype: int64

In [475]:
df_p[column].unique()

['More than 50%', '26 to 50%', '0 to 10%', '11 to 25%', NaN]
Categories (4, object): ['0 to 10%', '11 to 25%', '26 to 50%', 'More than 50%']

In [476]:
df_s[column].unique()

array([ 4.,  3.,  1.,  2., nan])

In [477]:
decoding_mapping = meta.value_labels[label]
decoding_mapping

{1.0: '0 to 10%',
 2.0: '11 to 25%',
 3.0: '26 to 50%',
 4.0: 'More than 50%',
 9.0: 'Omitted or invalid'}

Appling the above mapping to dataFrame `df_s` should reverse the encoding and give same column as in `df_p`.  Lets test this .... 

In [478]:
series_original = df_p.dropna()[column]
series_decoded = df_s.dropna()[column].map(decoding_mapping)

assert (series_original != series_decoded).sum()==0, f"Decoding encoded column {column} did not return original data!"

In conclusion, `df_p`, containing the original dataset (not encoded to numbers) probably has everything that we need.  So we can ignore `df_s`.  

The structure `meta` does have information that is useful, such as the description of columns, but we could get this from the codebook. 

So I'm going to use `pd.read_spss` to read in the datafiles.

## Cleaning

For each of the datafiles, that I plan to use, I will need to clean and save as a `feather` file.

Cleaning will involve:

* Remove columns that contain a single value. They tell me nothing.
* All ID columns start with `ID`? so they should be converted to `int`.
* Categorical variables that have categories that are `float` dtype should be converted to `float`.
* ... and on and on and on ...



## Load In Data

In [479]:
import pandas as pd
import pyreadstat

bsg_df, bsg_meta = pyreadstat.read_sav("orig/SPSS/bsgirlm8.sav")
bst_df, bsa_meta = pyreadstat.read_sav("orig/SPSS/bstirlm8.sav")

print("bsg_df shape:", bsg_df.shape)
print("bst_df shape:", bst_df.shape)

print(bsg_df.columns)

print(bst_df.columns)


bsg_df shape: (5090, 511)
bst_df shape: (9992, 122)
Index(['CTY', 'IDCNTRY', 'IDPOP', 'IDGRADER', 'IDGRADE', 'ITASSESS', 'IDBOOK',
       'IDSCHOOL', 'IDCLASS', 'IDSTUD',
       ...
       'BSDMLOWP', 'BSDSLOWP', 'BSDMZERP', 'BSDSZERP', 'BSDMCORP', 'BSDSCORP',
       'BSDMMISP', 'BSDSMISP', 'VERSION', 'SCOPE'],
      dtype='object', length=511)
Index(['CTY', 'IDCNTRY', 'IDPOP', 'IDGRADER', 'IDGRADE', 'ITASSESS', 'IDBOOK',
       'IDSCHOOL', 'IDCLASS', 'IDSTUD',
       ...
       'BSMIBM03', 'BSMIBM04', 'BSMIBM05', 'BSSIBM01', 'BSSIBM02', 'BSSIBM03',
       'BSSIBM04', 'BSSIBM05', 'VERSION', 'SCOPE'],
      dtype='object', length=122)


### Merging Both Data Sets

In [480]:
bsg_df.head(10)

,CTY,IDCNTRY,IDPOP,IDGRADER,IDGRADE,ITASSESS,IDBOOK,IDSCHOOL,IDCLASS,IDSTUD,ITSEX,BSDAGE,ITADMINI,ILRELIAB,ITLANG_SQ,LCID_SQ,BSBG01,BSBG03,BSBG04,BSBG05A,BSBG05B,BSBG05C,BSBG05D,BSBG05E,BSBG05F,BSBG05G,BSBG05H,BSBG05I,BSBG05J,BSBG06A,BSBG06B,BSBG07,BSBG08A,BSBG08B,BSBG09A,BSBG09B,BSBG10,BSBG11A,BSBG11B,BSBG12A,BSBG12B,BSBG12C,BSBG12D,BSBG12E,BSBG12F,BSBG13A,BSBG13B,BSBG13C,BSBG13D,BSBG13E,BSBG13F,BSBG13G,BSBG14A,BSBG14B,BSBG14C,BSBG14D,BSBG14E,BSBG14F,BSBG14G,BSBG14H,BSBG14I,BSBG15A,BSBG15B,BSBG15C,BSBG15D,BSBG15E,BSBG15F,BSBG16A,BSBG16B,BSBG16C,BSBG16D,BSBG16E,BSBG16F,BSBG16G,BSBG16H,BSBG17A,BSBG17B,BSBG17C,BSBG17D,BSBG17E,BSBG17F,BSBG17G,BSBG17H,BSBG17I,BSBG17J,BSBG17K,BSBG17L,BSBG17M,BSBG17N,BSBM18,BSBM19A,BSBM19B,BSBM19C,BSBM19D,BSBM19E,BSBM19F,BSBM19G,BSBM19H,BSBM19I,BSBM20A,BSBM20B,BSBM20C,BSBM20D,BSBM20E,BSBM20F,BSBM20G,BSBM20H,BSBM20I,BSBM20J,BSBM21A,BSBM21B,BSBM21C,BSBM21D,BSBM21E,BSBM21F,BSBM22A,BSBM22B,BSBM22C,BSBM22D,BSBM22E,BSBM22F,BSBM22G,BSBM22H,BSBM23A,BSBM23B,BSBM23C,BSBM23D,BSBM23E,BSBM23F,BSBM23G,BSBM23H,BSBM23I,BSBS24,BSBS25A,BSBS25B,BSBS25C,BSBS25D,BSBS25E,BSBS25F,BSBS25G,BSBS25H,BSBS25I,BSBS26A,BSBS26B,BSBS26C,BSBS26D,BSBS26E,BSBS26F,BSBS26G,BSBS26H,BSBS26I,BSBS26J,BSBS27A,BSBS27B,BSBS27C,BSBS27D,BSBS27E,BSBS27F,BSBS28A,BSBS28B,BSBS28C,BSBS28D,BSBS28E,BSBS28F,BSBS28G,BSBS28H,BSBS29A,BSBS29B,BSBS29C,BSBS29D,BSBS29E,BSBS29F,BSBS29G,BSBS29H,BSBS29I,BSBM30A,BSBS30B,BSBB24,BSBB25,BSBB26A,BSBB26B,BSBB26C,BSBB26D,BSBB26E,BSBB26F,BSBB26G,BSBB26H,BSBB26I,BSBB27A,BSBB27B,BSBB27C,BSBB27D,BSBB27E,BSBB27F,BSBB27G,BSBB27H,BSBB27I,BSBB27J,BSBB28A,BSBB28B,BSBB28C,BSBB28D,BSBB28E,BSBB28F,BSBB29A,BSBB29B,BSBB29C,BSBB29D,BSBB29E,BSBB29F,BSBB29G,BSBB29H,BSBE30,BSBE31,BSBE32A,BSBE32B,BSBE32C,BSBE32D,BSBE32E,BSBE32F,BSBE32G,BSBE32H,BSBE32I,BSBE33A,BSBE33B,BSBE33C,BSBE33D,BSBE33E,BSBE33F,BSBE33G,BSBE33H,BSBE33I,BSBE33J,BSBE34A,BSBE34B,BSBE34C,BSBE34D,BSBE34E,BSBE34F,BSBE35A,BSBE35B,BSBE35C,BSBE35D,BSBE35E,BSBE35F,BSBE35G,BSBE35H,BSBC36,BSBC37,BSBC38A,BSBC38B,BSBC38C,BSBC38D,BSBC38E,BSBC38F,BSBC38G,BSBC38H,BSBC38I,BSBC39A,BSBC39B,BSBC39C,BSBC39D,BSBC39E,BSBC39F,BSBC39G,BSBC39H,BSBC39I,BSBC39J,BSBC40A,BSBC40B,BSBC40C,BSBC40D,BSBC40E,BSBC40F,BSBC41A,BSBC41B,BSBC41C,BSBC41D,BSBC41E,BSBC41F,BSBC41G,BSBC41H,BSBP42,BSBP43,BSBP44A,BSBP44B,BSBP44C,BSBP44D,BSBP44E,BSBP44F,BSBP44G,BSBP44H,BSBP44I,BSBP45A,BSBP45B,BSBP45C,BSBP45D,BSBP45E,BSBP45F,BSBP45G,BSBP45H,BSBP45I,BSBP45J,BSBP46A,BSBP46B,BSBP46C,BSBP46D,BSBP46E,BSBP46F,BSBP47A,BSBP47B,BSBP47C,BSBP47D,BSBP47E,BSBP47F,BSBP47G,BSBP47H,BSBS48A,BSBS48B,BSBS48C,BSBS48D,BSBS48E,BSBS48F,BSBS48G,BSBS48H,BSBS48I,BSBM49A,BSBB49B,BSBE49C,BSBC49D,BSBP49E,JKZONE,JKREP,TOTWGT,HOUWGT,SENWGT,WGTFAC1,WGTADJ1,WGTFAC2,WGTADJ2,WGTFAC3,WGTADJ3,BSMMAT01,BSMMAT02,BSMMAT03,BSMMAT04,BSMMAT05,BSMNUM01,BSMNUM02,BSMNUM03,BSMNUM04,BSMNUM05,BSMALG01,BSMALG02,BSMALG03,BSMALG04,BSMALG05,BSMGEO01,BSMGEO02,BSMGEO03,BSMGEO04,BSMGEO05,BSMDAT01,BSMDAT02,BSMDAT03,BSMDAT04,BSMDAT05,BSMKNO01,BSMKNO02,BSMKNO03,BSMKNO04,BSMKNO05,BSMAPP01,BSMAPP02,BSMAPP03,BSMAPP04,BSMAPP05,BSMREA01,BSMREA02,BSMREA03,BSMREA04,BSMREA05,BSSSCI01,BSSSCI02,BSSSCI03,BSSSCI04,BSSSCI05,BSSBIO01,BSSBIO02,BSSBIO03,BSSBIO04,BSSBIO05,BSSCHE01,BSSCHE02,BSSCHE03,BSSCHE04,BSSCHE05,BSSPHY01,BSSPHY02,BSSPHY03,BSSPHY04,BSSPHY05,BSSEAR01,BSSEAR02,BSSEAR03,BSSEAR04,BSSEAR05,BSSKNO01,BSSKNO02,BSSKNO03,BSSKNO04,BSSKNO05,BSSAPP01,BSSAPP02,BSSAPP03,BSSAPP04,BSSAPP05,BSSREA01,BSSREA02,BSSREA03,BSSREA04,BSSREA05,BSSENV01,BSSENV02,BSSENV03,BSSENV04,BSSENV05,BSMIBM01,BSMIBM02,BSMIBM03,BSMIBM04,BSMIBM05,BSSIBM01,BSSIBM02,BSSIBM03,BSSIBM04,BSSIBM05,BSBGHER,BSDGHER,BSBGSSB,BSDGSSB,BSBGSB,BSDGSB,BSBGSLM,BSDGSLM,BSBGICM,BSDGICM,BSBGDML,BSDGDML,BSBGSCM,BSDGSCM,BSBGSVM,BSDGSVM,BSBGSLS,BSDGSLS,BSBGICS,BSDGICS,BSBGSCS,BSDGSCS,BSBGSVS,BSDGSVS,BSBGSLB,BSDGSLB,BSBGICB,BSDGICB,BSBGSCB,BSDGSCB,BSBGSLE,BSDGSLE,BSBGICE,BSDGICE,BSBGSCE,BSDGSCE,BSBGSLC,BSDGSLC,BSBGICC,BSDGICC,BSBGSCC,BSDGSCC,BSBGSLP,BSDGSLP,BSBGICP,BSDGICP,BSBGSCP,BSDGSCP,BSBGSEC,BSDGSEC,BSBGDSL,BSDGDSL,BSBGDBL,BSDGDBL,BSB

In [481]:
bst_df.head(10)

,CTY,IDCNTRY,IDPOP,IDGRADER,IDGRADE,ITASSESS,IDBOOK,IDSCHOOL,IDCLASS,IDSTUD,IDTEACH,IDLINK,IDTEALIN,IDSUBJ,ITCOURSE,MATSUBJ,SCISUBJ,NMTEACH,NSTEACH,NTEACH,JKZONE,JKREP,TCHWGT,MATWGT,SCIWGT,BSMMAT01,BSMMAT02,BSMMAT03,BSMMAT04,BSMMAT05,BSMNUM01,BSMNUM02,BSMNUM03,BSMNUM04,BSMNUM05,BSMALG01,BSMALG02,BSMALG03,BSMALG04,BSMALG05,BSMGEO01,BSMGEO02,BSMGEO03,BSMGEO04,BSMGEO05,BSMDAT01,BSMDAT02,BSMDAT03,BSMDAT04,BSMDAT05,BSMKNO01,BSMKNO02,BSMKNO03,BSMKNO04,BSMKNO05,BSMAPP01,BSMAPP02,BSMAPP03,BSMAPP04,BSMAPP05,BSMREA01,BSMREA02,BSMREA03,BSMREA04,BSMREA05,BSSSCI01,BSSSCI02,BSSSCI03,BSSSCI04,BSSSCI05,BSSBIO01,BSSBIO02,BSSBIO03,BSSBIO04,BSSBIO05,BSSCHE01,BSSCHE02,BSSCHE03,BSSCHE04,BSSCHE05,BSSPHY01,BSSPHY02,BSSPHY03,BSSPHY04,BSSPHY05,BSSEAR01,BSSEAR02,BSSEAR03,BSSEAR04,BSSEAR05,BSSKNO01,BSSKNO02,BSSKNO03,BSSKNO04,BSSKNO05,BSSAPP01,BSSAPP02,BSSAPP03,BSSAPP04,BSSAPP05,BSSREA01,BSSREA02,BSSREA03,BSSREA04,BSSREA05,BSSENV01,BSSENV02,BSSENV03,BSSENV04,BSSENV05,BSMIBM01,BSMIBM02,BSMIBM03,BSMIBM04,BSMIBM05,BSSIBM01,BSSIBM02,BSSIBM03,BSSIBM04,BSSIBM05,VERSION,SCOPE
0,IRL,372.0,2.0,2.0,8.0,1.0,1.0,5001.0,500101.0,50010101.0,500102.0,2.0,50010202.0,1.0,1.0,1.0,0.0,1.0,1.0,2.0,72.0,1.0,4.222826,8.445652,NaN,579.28271,619.61052,629.98681,635.76196,622.21290,616.15050,631.59033,586.11926,638.12500,662.16888,578.31469,623.23130,628.23201,621.59125,637.92312,587.19405,575.88615,594.31813,592.97973,579.86785,612.05582,627.74179,602.25184,623.19325,604.42856,650.54364,605.49470,606.24526,666.36207,619.27620,590.75504,595.06220,592.73813,636.12174,610.08847,621.33201,626.64541,627.60488,646.77978,623.79722,657.63011,627.23440,644.16322,645.77937,658.87835,657.00376,590.69615,638.94986,591.14361,629.62840,632.76343,656.30260,639.60886,587.37886,630.07117,626.14763,609.86176,626.04893,555.92748,610.96796,599.95029,611.03456,604.53424,537.31568,608.47505,672.10613,563.27978,621.39127,602.22245,628.14600,658.02641,577.68936,654.49987,606.37622,652.73081,640.20600,571.13373,616.09195,591.92681,575.99706,605.23698,640.92939,616.31810,554.18921,592.68977,4.0,4.0,5.0,5.0,4.0,5.0,5.0,5.0,5.0,5.0,4.0,1.0
1,IRL,372.0,2.0,2.0,8.0,1.0,1.0,5001.0,500101.0,50010101.0,500105.0,5.0,50010505.0,2.0,6.0,0.0,1.0,1.0,1.0,2.0,72.0,1.0,4.222826,NaN,8.445652,579.28271,619.61052,629.98681,635.76196,622.21290,616.15050,631.59033,586.11926,638.12500,662.16888,578.31469,623.23130,628.23201,621.59125,637.92312,587.19405,575.88615,594.31813,592.97973,579.86785,612.05582,627.74179,602.25184,623.19325,604.42856,650.54364,605.49470,606.24526,666.36207,619.27620,590.75504,595.06220,592.73813,636.12174,610.08847,621.33201,626.64541,627.60488,646.77978,623.79722,657.63011,627.23440,644.16322,645.77937,658.87835,657.00376,590.69615,638.94986,591.14361,629.62840,632.76343,656.30260,639.60886,587.37886,630.07117,626.14763,609.86176,626.04893,555.92748,610.96796,599.95029,611.03456,604.53424,537.31568,608.47505,672.10613,563.27978,621.39127,602.22245,628.14600,658.02641,577.68936,654.49987,606.37622,652.73081,640.20600,571.13373,616.09195,591.92681,575.99706,605.23698,640.92939,616.31810,554.18921,592.68977,4.0,4.0,5.0,5.0,4.0,5.0,5.0,5.0,5.0,5.0,4.0,1.0
2,IRL,372.0,2.0,2.0,8.0,1.0,9.0,5001.0,500101.0,50010102.0,500102.0,2.0,50010202.0,1.0,1.0,1.0,0.0,1.0,1.0,2.0,72.0,1.0,4.222826,8.445652,NaN,499.41713,562.87055,509.80525,540.81050,520.44525,522.25563,529.21710,514.47814,532.33674,546.25053,490.40324,491.33716,527.14997,529.86283,509.75121,521.60988,472.84961,539.24729,518.13734,526.18991,519.43046,506.46744,560.31565,600.85181,505.98160,601.07800,546.96674,624.88359,556.40946,581.19257,552.65827,547.96195,567.19392,541.69080,555.99936,533.73324,509.02002,580.64457,536.73935,545.32439,581.94834,558.02697,559.05058,604.61859,575.33909,517.67893,549.86388,543.18422,520.06890,552.73200,645.90156,630.17521,608.49764,609.10471,609.30590,627.66231,598.47498,584.46311,519.64423,572.02414,581.94751,600.51262,590.53285,629.05557,582.80864,563.32123,548.08971,544.38183,604.38335,608.33725,545.65519,567.11844,

In [482]:
merged_df = pd.merge(bsg_df, bst_df, on=['IDSTUD', 'IDCLASS', 'IDSCHOOL', 'CTY', 'IDCNTRY', 'IDGRADER', 'IDGRADE'], how='inner')
merged_df.columns

Index(['CTY', 'IDCNTRY', 'IDPOP_x', 'IDGRADER', 'IDGRADE', 'ITASSESS_x',
       'IDBOOK_x', 'IDSCHOOL', 'IDCLASS', 'IDSTUD',
       ...
       'BSMIBM03_y', 'BSMIBM04_y', 'BSMIBM05_y', 'BSSIBM01_y', 'BSSIBM02_y',
       'BSSIBM03_y', 'BSSIBM04_y', 'BSSIBM05_y', 'VERSION_y', 'SCOPE_y'],
      dtype='object', length=626)

In [483]:
merged_df.describe()


,IDCNTRY,IDPOP_x,IDGRADER,IDGRADE,ITASSESS_x,IDBOOK_x,IDSCHOOL,IDCLASS,IDSTUD,ITSEX,BSDAGE,ITADMINI,ILRELIAB,ITLANG_SQ,LCID_SQ,BSBG01,BSBG03,BSBG04,BSBG05A,BSBG05B,BSBG05C,BSBG05D,BSBG05E,BSBG05F,BSBG05G,BSBG05H,BSBG05I,BSBG05J,BSBG06A,BSBG06B,BSBG07,BSBG08A,BSBG08B,BSBG09A,BSBG09B,BSBG10,BSBG11A,BSBG11B,BSBG12A,BSBG12B,BSBG12C,BSBG12D,BSBG12E,BSBG12F,BSBG13A,BSBG13B,BSBG13C,BSBG13D,BSBG13E,BSBG13F,BSBG13G,BSBG14A,BSBG14B,BSBG14C,BSBG14D,BSBG14E,BSBG14F,BSBG14G,BSBG14H,BSBG14I,BSBG15A,BSBG15B,BSBG15C,BSBG15D,BSBG15E,BSBG15F,BSBG16A,BSBG16B,BSBG16C,BSBG16D,BSBG16E,BSBG16F,BSBG16G,BSBG16H,BSBG17A,BSBG17B,BSBG17C,BSBG17D,BSBG17E,BSBG17F,BSBG17G,BSBG17H,BSBG17I,BSBG17J,BSBG17K,BSBG17L,BSBG17M,BSBG17N,BSBM18,BSBM19A,BSBM19B,BSBM19C,BSBM19D,BSBM19E,BSBM19F,BSBM19G,BSBM19H,BSBM19I,BSBM20A,BSBM20B,BSBM20C,BSBM20D,BSBM20E,BSBM20F,BSBM20G,BSBM20H,BSBM20I,BSBM20J,BSBM21A,BSBM21B,BSBM21C,BSBM21D,BSBM21E,BSBM21F,BSBM22A,BSBM22B,BSBM22C,BSBM22D,BSBM22E,BSBM22F,BSBM22G,BSBM22H,BSBM23A,BSBM23B,BSBM23C,BSBM23D,BSBM23E,BSBM23F,BSBM23G,BSBM23H,BSBM23I,BSBS24,BSBS25A,BSBS25B,BSBS25C,BSBS25D,BSBS25E,BSBS25F,BSBS25G,BSBS25H,BSBS25I,BSBS26A,BSBS26B,BSBS26C,BSBS26D,BSBS26E,BSBS26F,BSBS26G,BSBS26H,BSBS26I,BSBS26J,BSBS27A,BSBS27B,BSBS27C,BSBS27D,BSBS27E,BSBS27F,BSBS28A,BSBS28B,BSBS28C,BSBS28D,BSBS28E,BSBS28F,BSBS28G,BSBS28H,BSBS29A,BSBS29B,BSBS29C,BSBS29D,BSBS29E,BSBS29F,BSBS29G,BSBS29H,BSBS29I,BSBM30A,BSBS30B,BSBB24,BSBB25,BSBB26A,BSBB26B,BSBB26C,BSBB26D,BSBB26E,BSBB26F,BSBB26G,BSBB26H,BSBB26I,BSBB27A,BSBB27B,BSBB27C,BSBB27D,BSBB27E,BSBB27F,BSBB27G,BSBB27H,BSBB27I,BSBB27J,BSBB28A,BSBB28B,BSBB28C,BSBB28D,BSBB28E,BSBB28F,BSBB29A,BSBB29B,BSBB29C,BSBB29D,BSBB29E,BSBB29F,BSBB29G,BSBB29H,BSBE30,BSBE31,BSBE32A,BSBE32B,BSBE32C,BSBE32D,BSBE32E,BSBE32F,BSBE32G,BSBE32H,BSBE32I,BSBE33A,BSBE33B,BSBE33C,BSBE33D,BSBE33E,BSBE33F,BSBE33G,BSBE33H,BSBE33I,BSBE33J,BSBE34A,BSBE34B,BSBE34C,BSBE34D,BSBE34E,BSBE34F,BSBE35A,BSBE35B,BSBE35C,BSBE35D,BSBE35E,BSBE35F,BSBE35G,BSBE35H,BSBC36,BSBC37,BSBC38A,BSBC38B,BSBC38C,BSBC38D,BSBC38E,BSBC38F,BSBC38G,BSBC38H,BSBC38I,BSBC39A,BSBC39B,BSBC39C,BSBC39D,BSBC39E,BSBC39F,BSBC39G,BSBC39H,BSBC39I,BSBC39J,BSBC40A,BSBC40B,BSBC40C,BSBC40D,BSBC40E,BSBC40F,BSBC41A,BSBC41B,BSBC41C,BSBC41D,BSBC41E,BSBC41F,BSBC41G,BSBC41H,BSBP42,BSBP43,BSBP44A,BSBP44B,BSBP44C,BSBP44D,BSBP44E,BSBP44F,BSBP44G,BSBP44H,BSBP44I,BSBP45A,BSBP45B,BSBP45C,BSBP45D,BSBP45E,BSBP45F,BSBP45G,BSBP45H,BSBP45I,BSBP45J,BSBP46A,BSBP46B,BSBP46C,BSBP46D,BSBP46E,BSBP46F,BSBP47A,BSBP47B,BSBP47C,BSBP47D,BSBP47E,BSBP47F,BSBP47G,BSBP47H,BSBS48A,BSBS48B,BSBS48C,BSBS48D,BSBS48E,BSBS48F,BSBS48G,BSBS48H,BSBS48I,BSBM49A,BSBB49B,BSBE49C,BSBC49D,BSBP49E,JKZONE_x,JKREP_x,TOTWGT,HOUWGT,SENWGT,WGTFAC1,WGTADJ1,WGTFAC2,WGTADJ2,WGTFAC3,WGTADJ3,BSMMAT01_x,BSMMAT02_x,BSMMAT03_x,BSMMAT04_x,BSMMAT05_x,BSMNUM01_x,BSMNUM02_x,BSMNUM03_x,BSMNUM04_x,BSMNUM05_x,BSMALG01_x,BSMALG02_x,BSMALG03_x,BSMALG04_x,BSMALG05_x,BSMGEO01_x,BSMGEO02_x,BSMGEO03_x,BSMGEO04_x,BSMGEO05_x,BSMDAT01_x,BSMDAT02_x,BSMDAT03_x,BSMDAT04_x,BSMDAT05_x,BSMKNO01_x,BSMKNO02_x,BSMKNO03_x,BSMKNO04_x,BSMKNO05_x,BSMAPP01_x,BSMAPP02_x,BSMAPP03_x,BSMAPP04_x,BSMAPP05_x,BSMREA01_x,BSMREA02_x,BSMREA03_x,BSMREA04_x,BSMREA05_x,BSSSCI01_x,BSSSCI02_x,BSSSCI03_x,BSSSCI04_x,BSSSCI05_x,BSSBIO01_x,BSSBIO02_x,BSSBIO03_x,BSSBIO04_x,BSSBIO05_x,BSSCHE01_x,BSSCHE02_x,BSSCHE03_x,BSSCHE04_x,BSSCHE05_x,BSSPHY01_x,BSSPHY02_x,BSSPHY03_x,BSSPHY04_x,BSSPHY05_x,BSSEAR01_x,BSSEAR02_x,BSSEAR03_x,BSSEAR04_x,BSSEAR05_x,BSSKNO01_x,BSSKNO02_x,BSSKNO03_x,BSSKNO04_x,BSSKNO05_x,BSSAPP01_x,BSSAPP02_x,BSSAPP03_x,BSSAPP04_x,BSSAPP05_x,BSSREA01_x,BSSREA02_x,BSSREA03_x,BSSREA04_x,BSSREA05_x,BSSENV01_x,BSSENV02_x,BSSENV03_x,BSSENV04_x,BSSENV05_x,BSMIBM01_x,BSMIBM02_x,BSMIBM03_x,BSMIBM04_x,BSMIBM05_x,BSSIBM01_x,BSSIBM02_x,BSSIBM03_x,BSSIBM04_x,BSSIBM05_x,BSBGHER,BSDGHER,BSBGSSB,BSDGSSB,BSBGSB,BSDGSB,BSBGSLM,BSDGSLM,BSBGICM,BSDGICM,BSBGDML,BSDGDML,BSBGSCM,BSDGSCM,BSBGSVM,BSDGSVM,BSBGSLS,BSDGSLS,BSBGICS,BSDGICS,BSBGSCS,BSDGSCS,BSBGSVS,BSDGSVS,BSBGSLB,BSDGSLB,BSBGICB,BSDGICB,BSBGSCB,BSDGSCB

In [484]:
merged_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9992 entries, 0 to 9991
Columns: 626 entries, CTY to SCOPE_y
dtypes: float64(625), object(1)
memory usage: 47.7+ MB


In [485]:
merged_df.isnull().sum()


CTY           0
IDCNTRY       0
IDPOP_x       0
IDGRADER      0
IDGRADE       0
             ..
BSSIBM03_y    0
BSSIBM04_y    0
BSSIBM05_y    0
VERSION_y     0
SCOPE_y       0
Length: 626, dtype: int64

In [486]:
merged_df['MATWGT'] = merged_df['MATWGT'].fillna(merged_df['MATWGT'].mean())
merged_df['SCIWGT'] = merged_df['SCIWGT'].fillna(merged_df['SCIWGT'].mean())
merged_df.isnull().sum()

CTY           0
IDCNTRY       0
IDPOP_x       0
IDGRADER      0
IDGRADE       0
             ..
BSSIBM03_y    0
BSSIBM04_y    0
BSSIBM05_y    0
VERSION_y     0
SCOPE_y       0
Length: 626, dtype: int64

In [487]:
merged_df.nunique()


CTY           1
IDCNTRY       1
IDPOP_x       1
IDGRADER      1
IDGRADE       1
             ..
BSSIBM03_y    5
BSSIBM04_y    5
BSSIBM05_y    5
VERSION_y     1
SCOPE_y       1
Length: 626, dtype: int64

In [488]:
merged_df.duplicated().sum()


0

In [489]:
# Drop columns with more than 50% NaN values
threshold = len(merged_df) * 0.5
merged_df = merged_df.dropna(thresh=threshold, axis=1)

merged_df.head(10)


,CTY,IDCNTRY,IDPOP_x,IDGRADER,IDGRADE,ITASSESS_x,IDBOOK_x,IDSCHOOL,IDCLASS,IDSTUD,ITSEX,BSDAGE,ITADMINI,ILRELIAB,ITLANG_SQ,LCID_SQ,BSBG01,BSBG03,BSBG04,BSBG05A,BSBG05B,BSBG05C,BSBG05D,BSBG05E,BSBG05F,BSBG05G,BSBG05H,BSBG05I,BSBG05J,BSBG06A,BSBG06B,BSBG07,BSBG08A,BSBG08B,BSBG09A,BSBG10,BSBG11A,BSBG11B,BSBG12A,BSBG12B,BSBG12C,BSBG12D,BSBG12E,BSBG12F,BSBG13A,BSBG13B,BSBG13C,BSBG13D,BSBG13E,BSBG13F,BSBG13G,BSBG14A,BSBG14B,BSBG14C,BSBG14D,BSBG14E,BSBG14F,BSBG14G,BSBG14H,BSBG14I,BSBG15A,BSBG15B,BSBG15C,BSBG15D,BSBG15E,BSBG15F,BSBG16A,BSBG16B,BSBG16C,BSBG16D,BSBG16E,BSBG16F,BSBG16G,BSBG16H,BSBG17A,BSBG17B,BSBG17C,BSBG17D,BSBG17E,BSBG17F,BSBG17G,BSBG17H,BSBG17I,BSBG17J,BSBG17K,BSBG17L,BSBG17M,BSBG17N,BSBM18,BSBM19A,BSBM19B,BSBM19C,BSBM19D,BSBM19E,BSBM19F,BSBM19G,BSBM19H,BSBM19I,BSBM20A,BSBM20B,BSBM20C,BSBM20D,BSBM20E,BSBM20F,BSBM20G,BSBM20H,BSBM20I,BSBM20J,BSBM21A,BSBM21B,BSBM21C,BSBM21D,BSBM21E,BSBM21F,BSBM22A,BSBM22B,BSBM22C,BSBM22D,BSBM22E,BSBM22F,BSBM22G,BSBM22H,BSBM23A,BSBM23B,BSBM23C,BSBM23D,BSBM23E,BSBM23F,BSBM23G,BSBM23H,BSBM23I,BSBS24,BSBS25A,BSBS25B,BSBS25C,BSBS25D,BSBS25E,BSBS25F,BSBS25G,BSBS25H,BSBS25I,BSBS26A,BSBS26B,BSBS26C,BSBS26D,BSBS26E,BSBS26F,BSBS26G,BSBS26H,BSBS26I,BSBS26J,BSBS27A,BSBS27B,BSBS27C,BSBS27D,BSBS27E,BSBS27F,BSBS28A,BSBS28B,BSBS28C,BSBS28D,BSBS28E,BSBS28F,BSBS28G,BSBS28H,BSBS29A,BSBS29B,BSBS29C,BSBS29D,BSBS29E,BSBS29F,BSBS29G,BSBS29H,BSBS29I,BSBM30A,BSBS30B,BSBS48A,BSBS48B,BSBS48C,BSBS48D,BSBS48E,BSBS48F,BSBS48G,BSBS48H,BSBS48I,BSBM49A,JKZONE_x,JKREP_x,TOTWGT,HOUWGT,SENWGT,WGTFAC1,WGTADJ1,WGTFAC2,WGTADJ2,WGTFAC3,WGTADJ3,BSMMAT01_x,BSMMAT02_x,BSMMAT03_x,BSMMAT04_x,BSMMAT05_x,BSMNUM01_x,BSMNUM02_x,BSMNUM03_x,BSMNUM04_x,BSMNUM05_x,BSMALG01_x,BSMALG02_x,BSMALG03_x,BSMALG04_x,BSMALG05_x,BSMGEO01_x,BSMGEO02_x,BSMGEO03_x,BSMGEO04_x,BSMGEO05_x,BSMDAT01_x,BSMDAT02_x,BSMDAT03_x,BSMDAT04_x,BSMDAT05_x,BSMKNO01_x,BSMKNO02_x,BSMKNO03_x,BSMKNO04_x,BSMKNO05_x,BSMAPP01_x,BSMAPP02_x,BSMAPP03_x,BSMAPP04_x,BSMAPP05_x,BSMREA01_x,BSMREA02_x,BSMREA03_x,BSMREA04_x,BSMREA05_x,BSSSCI01_x,BSSSCI02_x,BSSSCI03_x,BSSSCI04_x,BSSSCI05_x,BSSBIO01_x,BSSBIO02_x,BSSBIO03_x,BSSBIO04_x,BSSBIO05_x,BSSCHE01_x,BSSCHE02_x,BSSCHE03_x,BSSCHE04_x,BSSCHE05_x,BSSPHY01_x,BSSPHY02_x,BSSPHY03_x,BSSPHY04_x,BSSPHY05_x,BSSEAR01_x,BSSEAR02_x,BSSEAR03_x,BSSEAR04_x,BSSEAR05_x,BSSKNO01_x,BSSKNO02_x,BSSKNO03_x,BSSKNO04_x,BSSKNO05_x,BSSAPP01_x,BSSAPP02_x,BSSAPP03_x,BSSAPP04_x,BSSAPP05_x,BSSREA01_x,BSSREA02_x,BSSREA03_x,BSSREA04_x,BSSREA05_x,BSSENV01_x,BSSENV02_x,BSSENV03_x,BSSENV04_x,BSSENV05_x,BSMIBM01_x,BSMIBM02_x,BSMIBM03_x,BSMIBM04_x,BSMIBM05_x,BSSIBM01_x,BSSIBM02_x,BSSIBM03_x,BSSIBM04_x,BSSIBM05_x,BSBGHER,BSDGHER,BSBGSSB,BSDGSSB,BSBGSB,BSDGSB,BSBGSLM,BSDGSLM,BSBGICM,BSDGICM,BSBGDML,BSDGDML,BSBGSCM,BSDGSCM,BSBGSVM,BSDGSVM,BSBGSLS,BSDGSLS,BSBGICS,BSDGICS,BSBGSCS,BSDGSCS,BSBGSVS,BSDGSVS,BSBGSEC,BSDGSEC,BSBGDSL,BSDGDSL,BSBGVEP,BSDGVEP,BSDG05S,BSDGEDUP,BSDMLOWP,BSDSLOWP,BSDMZERP,BSDSZERP,BSDMCORP,BSDSCORP,BSDMMISP,BSDSMISP,VERSION_x,SCOPE_x,IDPOP_y,ITASSESS_y,IDBOOK_y,IDTEACH,IDLINK,IDTEALIN,IDSUBJ,ITCOURSE,MATSUBJ,SCISUBJ,NMTEACH,NSTEACH,NTEACH,JKZONE_y,JKREP_y,TCHWGT,MATWGT,SCIWGT,BSMMAT01_y,BSMMAT02_y,BSMMAT03_y,BSMMAT04_y,BSMMAT05_y,BSMNUM01_y,BSMNUM02_y,BSMNUM03_y,BSMNUM04_y,BSMNUM05_y,BSMALG01_y,BSMALG02_y,BSMALG03_y,BSMALG04_y,BSMALG05_y,BSMGEO01_y,BSMGEO02_y,BSMGEO03_y,BSMGEO04_y,BSMGEO05_y,BSMDAT01_y,BSMDAT02_y,BSMDAT03_y,BSMDAT04_y,BSMDAT05_y,BSMKNO01_y,BSMKNO02_y,BSMKNO03_y,BSMKNO04_y,BSMKNO05_y,BSMAPP01_y,BSMAPP02_y,BSMAPP03_y,BSMAPP04_y,BSMAPP05_y,BSMREA01_y,BSMREA02_y,BSMREA03_y,BSMREA04_y,BSMREA05_y,BSSSCI01_y,BSSSCI02_y,BSSSCI03_y,BSSSCI04_y,BSSSCI05_y,BSSBIO01_y,BSSBIO02_y,BSSBIO03_y,BSSBIO04_y,BSSBIO05_y,BSSCHE01_y,BSSCHE02_y,BSSCHE03_y,BSSCHE04_y,BSSCHE05_y,BSSPHY01_y,BSSPHY02_y,BSSPHY03_y,BSSPHY04_y,BSSPHY05_y,BSSEAR01_y,BSSEAR02_y,BSSEAR03_y,BSSEAR04_y,BSSEAR05_y,BSSKNO01_y,BSSKNO02_y,BSSKNO03_y,BSSKNO04_y,BSSKNO05_y,BSSAPP01_y,BSSAPP02_y,BSSAPP03_y,BSSAPP04_y,BSSAPP05_y,BSSREA01_y,BSSREA02_y,BSSREA03_y,BSSREA04_y,BSSREA05_y,BSSENV01_y,BSSE

## Tidy Up Duplicates And Naming

In [490]:
# Drop Duplicate Columns
columns_to_drop = [col for col in merged_df.columns if col.endswith('_y')]
merged_df = merged_df.drop(columns=columns_to_drop)
# Remove _x from column names
merged_df.columns = merged_df.columns.str.replace('_x', '')
merged_df.head(10)

,CTY,IDCNTRY,IDPOP,IDGRADER,IDGRADE,ITASSESS,IDBOOK,IDSCHOOL,IDCLASS,IDSTUD,ITSEX,BSDAGE,ITADMINI,ILRELIAB,ITLANG_SQ,LCID_SQ,BSBG01,BSBG03,BSBG04,BSBG05A,BSBG05B,BSBG05C,BSBG05D,BSBG05E,BSBG05F,BSBG05G,BSBG05H,BSBG05I,BSBG05J,BSBG06A,BSBG06B,BSBG07,BSBG08A,BSBG08B,BSBG09A,BSBG10,BSBG11A,BSBG11B,BSBG12A,BSBG12B,BSBG12C,BSBG12D,BSBG12E,BSBG12F,BSBG13A,BSBG13B,BSBG13C,BSBG13D,BSBG13E,BSBG13F,BSBG13G,BSBG14A,BSBG14B,BSBG14C,BSBG14D,BSBG14E,BSBG14F,BSBG14G,BSBG14H,BSBG14I,BSBG15A,BSBG15B,BSBG15C,BSBG15D,BSBG15E,BSBG15F,BSBG16A,BSBG16B,BSBG16C,BSBG16D,BSBG16E,BSBG16F,BSBG16G,BSBG16H,BSBG17A,BSBG17B,BSBG17C,BSBG17D,BSBG17E,BSBG17F,BSBG17G,BSBG17H,BSBG17I,BSBG17J,BSBG17K,BSBG17L,BSBG17M,BSBG17N,BSBM18,BSBM19A,BSBM19B,BSBM19C,BSBM19D,BSBM19E,BSBM19F,BSBM19G,BSBM19H,BSBM19I,BSBM20A,BSBM20B,BSBM20C,BSBM20D,BSBM20E,BSBM20F,BSBM20G,BSBM20H,BSBM20I,BSBM20J,BSBM21A,BSBM21B,BSBM21C,BSBM21D,BSBM21E,BSBM21F,BSBM22A,BSBM22B,BSBM22C,BSBM22D,BSBM22E,BSBM22F,BSBM22G,BSBM22H,BSBM23A,BSBM23B,BSBM23C,BSBM23D,BSBM23E,BSBM23F,BSBM23G,BSBM23H,BSBM23I,BSBS24,BSBS25A,BSBS25B,BSBS25C,BSBS25D,BSBS25E,BSBS25F,BSBS25G,BSBS25H,BSBS25I,BSBS26A,BSBS26B,BSBS26C,BSBS26D,BSBS26E,BSBS26F,BSBS26G,BSBS26H,BSBS26I,BSBS26J,BSBS27A,BSBS27B,BSBS27C,BSBS27D,BSBS27E,BSBS27F,BSBS28A,BSBS28B,BSBS28C,BSBS28D,BSBS28E,BSBS28F,BSBS28G,BSBS28H,BSBS29A,BSBS29B,BSBS29C,BSBS29D,BSBS29E,BSBS29F,BSBS29G,BSBS29H,BSBS29I,BSBM30A,BSBS30B,BSBS48A,BSBS48B,BSBS48C,BSBS48D,BSBS48E,BSBS48F,BSBS48G,BSBS48H,BSBS48I,BSBM49A,JKZONE,JKREP,TOTWGT,HOUWGT,SENWGT,WGTFAC1,WGTADJ1,WGTFAC2,WGTADJ2,WGTFAC3,WGTADJ3,BSMMAT01,BSMMAT02,BSMMAT03,BSMMAT04,BSMMAT05,BSMNUM01,BSMNUM02,BSMNUM03,BSMNUM04,BSMNUM05,BSMALG01,BSMALG02,BSMALG03,BSMALG04,BSMALG05,BSMGEO01,BSMGEO02,BSMGEO03,BSMGEO04,BSMGEO05,BSMDAT01,BSMDAT02,BSMDAT03,BSMDAT04,BSMDAT05,BSMKNO01,BSMKNO02,BSMKNO03,BSMKNO04,BSMKNO05,BSMAPP01,BSMAPP02,BSMAPP03,BSMAPP04,BSMAPP05,BSMREA01,BSMREA02,BSMREA03,BSMREA04,BSMREA05,BSSSCI01,BSSSCI02,BSSSCI03,BSSSCI04,BSSSCI05,BSSBIO01,BSSBIO02,BSSBIO03,BSSBIO04,BSSBIO05,BSSCHE01,BSSCHE02,BSSCHE03,BSSCHE04,BSSCHE05,BSSPHY01,BSSPHY02,BSSPHY03,BSSPHY04,BSSPHY05,BSSEAR01,BSSEAR02,BSSEAR03,BSSEAR04,BSSEAR05,BSSKNO01,BSSKNO02,BSSKNO03,BSSKNO04,BSSKNO05,BSSAPP01,BSSAPP02,BSSAPP03,BSSAPP04,BSSAPP05,BSSREA01,BSSREA02,BSSREA03,BSSREA04,BSSREA05,BSSENV01,BSSENV02,BSSENV03,BSSENV04,BSSENV05,BSMIBM01,BSMIBM02,BSMIBM03,BSMIBM04,BSMIBM05,BSSIBM01,BSSIBM02,BSSIBM03,BSSIBM04,BSSIBM05,BSBGHER,BSDGHER,BSBGSSB,BSDGSSB,BSBGSB,BSDGSB,BSBGSLM,BSDGSLM,BSBGICM,BSDGICM,BSBGDML,BSDGDML,BSBGSCM,BSDGSCM,BSBGSVM,BSDGSVM,BSBGSLS,BSDGSLS,BSBGICS,BSDGICS,BSBGSCS,BSDGSCS,BSBGSVS,BSDGSVS,BSBGSEC,BSDGSEC,BSBGDSL,BSDGDSL,BSBGVEP,BSDGVEP,BSDG05S,BSDGEDUP,BSDMLOWP,BSDSLOWP,BSDMZERP,BSDSZERP,BSDMCORP,BSDSCORP,BSDMMISP,BSDSMISP,VERSION,SCOPE,IDTEACH,IDLINK,IDTEALIN,IDSUBJ,ITCOURSE,MATSUBJ,SCISUBJ,NMTEACH,NSTEACH,NTEACH,TCHWGT,MATWGT,SCIWGT
0,IRL,372.0,2.0,2.0,8.0,1.0,1.0,5001.0,500101.0,50010101.0,1.0,14.17,3.0,1.0,1.0,6153.0,1.0,3.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,8.0,8.0,6.0,2.0,2.0,1.0,5.0,3.0,3.0,1.0,1.0,3.0,3.0,2.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,3.0,3.0,3.0,3.0,3.0,1.0,3.0,1.0,1.0,1.0,3.0,3.0,3.0,3.0,2.0,2.0,2.0,3.0,2.0,1.0,2.0,2.0,3.0,1.0,2.0,1.0,1.0,2.0,3.0,4.0,3.0,4.0,3.0,2.0,2.0,3.0,1.0,3.0,1.0,1.0,2.0,4.0,3.0,2.0,4.0,3.0,3.0,3.0,2.0,2.0,3.0,4.0,2.0,4.0,1.0,3.0,1.0,3.0,3.0,2.0,2.0,2.0,2.0,3.0,2.0,3.0,3.0,3.0,3.0,2.0,2.0,2.0,1.0,1.0,2.0,3.0,2.0,1.0,2.0,2.0,1.0,4.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,2.0,1.0,3.0,4.0,3.0,3.0,2.0,2.0,2.0,3.0,3.0,2.0,2.0,4.0,4.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,72.0,1.0,8.445652,0.601726,0.059109,4.625,1.0,1.5,1.0,1.0,1.217391,579.28271,619.61052,629.98681,635.76196,622.21290,616.15050,631.59033,586.11926,638.12500,662.16888,578.31469,623.23130,628.23201,621.59125,637.92312,587.19405,575.88615,594.31813,592.97973,579.86785,612.05582,627.74179,602.25184,623.19325,604.42856,650.54364,605.49470,606.24526,666.36207,619.27620,590.75504

### Drop All Science Related Columns

In [491]:
columns_to_drop = [col for col in merged_df.columns if col.startswith(("BSBS", "BSBB", "BSBE", "BSBC", "BSBP", "BSSB", "BSSS", "BSSC", "BSSP", "BSSE", "BSSR", "BSSA", "BSSK"))]

merged_df = merged_df.drop(columns=columns_to_drop)
print(merged_df.shape)
merged_df.head(10)

(9992, 249)


,CTY,IDCNTRY,IDPOP,IDGRADER,IDGRADE,ITASSESS,IDBOOK,IDSCHOOL,IDCLASS,IDSTUD,ITSEX,BSDAGE,ITADMINI,ILRELIAB,ITLANG_SQ,LCID_SQ,BSBG01,BSBG03,BSBG04,BSBG05A,BSBG05B,BSBG05C,BSBG05D,BSBG05E,BSBG05F,BSBG05G,BSBG05H,BSBG05I,BSBG05J,BSBG06A,BSBG06B,BSBG07,BSBG08A,BSBG08B,BSBG09A,BSBG10,BSBG11A,BSBG11B,BSBG12A,BSBG12B,BSBG12C,BSBG12D,BSBG12E,BSBG12F,BSBG13A,BSBG13B,BSBG13C,BSBG13D,BSBG13E,BSBG13F,BSBG13G,BSBG14A,BSBG14B,BSBG14C,BSBG14D,BSBG14E,BSBG14F,BSBG14G,BSBG14H,BSBG14I,BSBG15A,BSBG15B,BSBG15C,BSBG15D,BSBG15E,BSBG15F,BSBG16A,BSBG16B,BSBG16C,BSBG16D,BSBG16E,BSBG16F,BSBG16G,BSBG16H,BSBG17A,BSBG17B,BSBG17C,BSBG17D,BSBG17E,BSBG17F,BSBG17G,BSBG17H,BSBG17I,BSBG17J,BSBG17K,BSBG17L,BSBG17M,BSBG17N,BSBM18,BSBM19A,BSBM19B,BSBM19C,BSBM19D,BSBM19E,BSBM19F,BSBM19G,BSBM19H,BSBM19I,BSBM20A,BSBM20B,BSBM20C,BSBM20D,BSBM20E,BSBM20F,BSBM20G,BSBM20H,BSBM20I,BSBM20J,BSBM21A,BSBM21B,BSBM21C,BSBM21D,BSBM21E,BSBM21F,BSBM22A,BSBM22B,BSBM22C,BSBM22D,BSBM22E,BSBM22F,BSBM22G,BSBM22H,BSBM23A,BSBM23B,BSBM23C,BSBM23D,BSBM23E,BSBM23F,BSBM23G,BSBM23H,BSBM23I,BSBM30A,BSBM49A,JKZONE,JKREP,TOTWGT,HOUWGT,SENWGT,WGTFAC1,WGTADJ1,WGTFAC2,WGTADJ2,WGTFAC3,WGTADJ3,BSMMAT01,BSMMAT02,BSMMAT03,BSMMAT04,BSMMAT05,BSMNUM01,BSMNUM02,BSMNUM03,BSMNUM04,BSMNUM05,BSMALG01,BSMALG02,BSMALG03,BSMALG04,BSMALG05,BSMGEO01,BSMGEO02,BSMGEO03,BSMGEO04,BSMGEO05,BSMDAT01,BSMDAT02,BSMDAT03,BSMDAT04,BSMDAT05,BSMKNO01,BSMKNO02,BSMKNO03,BSMKNO04,BSMKNO05,BSMAPP01,BSMAPP02,BSMAPP03,BSMAPP04,BSMAPP05,BSMREA01,BSMREA02,BSMREA03,BSMREA04,BSMREA05,BSMIBM01,BSMIBM02,BSMIBM03,BSMIBM04,BSMIBM05,BSSIBM01,BSSIBM02,BSSIBM03,BSSIBM04,BSSIBM05,BSBGHER,BSDGHER,BSBGSSB,BSDGSSB,BSBGSB,BSDGSB,BSBGSLM,BSDGSLM,BSBGICM,BSDGICM,BSBGDML,BSDGDML,BSBGSCM,BSDGSCM,BSBGSVM,BSDGSVM,BSBGSLS,BSDGSLS,BSBGICS,BSDGICS,BSBGSCS,BSDGSCS,BSBGSVS,BSDGSVS,BSBGSEC,BSDGSEC,BSBGDSL,BSDGDSL,BSBGVEP,BSDGVEP,BSDG05S,BSDGEDUP,BSDMLOWP,BSDSLOWP,BSDMZERP,BSDSZERP,BSDMCORP,BSDSCORP,BSDMMISP,BSDSMISP,VERSION,SCOPE,IDTEACH,IDLINK,IDTEALIN,IDSUBJ,ITCOURSE,MATSUBJ,SCISUBJ,NMTEACH,NSTEACH,NTEACH,TCHWGT,MATWGT,SCIWGT
0,IRL,372.0,2.0,2.0,8.0,1.0,1.0,5001.0,500101.0,50010101.0,1.0,14.17,3.0,1.0,1.0,6153.0,1.0,3.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,8.0,8.0,6.0,2.0,2.0,1.0,5.0,3.0,3.0,1.0,1.0,3.0,3.0,2.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,3.0,3.0,3.0,3.0,3.0,1.0,3.0,1.0,1.0,1.0,3.0,3.0,3.0,3.0,2.0,2.0,2.0,3.0,2.0,1.0,2.0,2.0,3.0,1.0,2.0,1.0,1.0,2.0,3.0,4.0,3.0,4.0,3.0,2.0,2.0,3.0,1.0,3.0,1.0,1.0,2.0,4.0,3.0,2.0,4.0,3.0,3.0,3.0,2.0,2.0,3.0,4.0,2.0,4.0,1.0,3.0,1.0,3.0,3.0,2.0,2.0,2.0,2.0,3.0,2.0,3.0,3.0,3.0,3.0,2.0,2.0,2.0,1.0,1.0,2.0,3.0,2.0,1.0,2.0,1.0,1.0,72.0,1.0,8.445652,0.601726,0.059109,4.625,1.0,1.5,1.0,1.0,1.217391,579.28271,619.61052,629.98681,635.76196,622.21290,616.15050,631.59033,586.11926,638.12500,662.16888,578.31469,623.23130,628.23201,621.59125,637.92312,587.19405,575.88615,594.31813,592.97973,579.86785,612.05582,627.74179,602.25184,623.19325,604.42856,650.54364,605.49470,606.24526,666.36207,619.27620,590.75504,595.06220,592.73813,636.12174,610.08847,621.33201,626.64541,627.60488,646.77978,623.79722,4.0,4.0,5.0,5.0,4.0,5.0,5.0,5.0,5.0,5.0,11.16547,2.0,9.23501,2.0,7.83267,3.0,9.08471,3.0,7.71297,3.0,9.05919,2.0,9.72439,3.0,9.19823,2.0,12.68366,1.0,12.24370,1.0,10.19911,2.0,12.44879,1.0,12.74515,1.0,9.83635,2.0,8.43770,2.0,3.0,6.0,2.0,2.0,2.0,2.0,72.72727,72.72727,0.00000,0.00000,4.0,1.0,500102.0,2.0,50010202.0,1.0,1.0,1.0,0.0,1.0,1.0,2.0,4.222826,8.445652,14.087339
1,IRL,372.0,2.0,2.0,8.0,1.0,1.0,5001.0,500101.0,50010101.0,1.0,14.17,3.0,1.0,1.0,6153.0,1.0,3.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,8.0,8.0,6.0,2.0,2.0,1.0,5.0,3.0,3.0,1.0,1.0,3.0,3.0,2.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,3.0,3.0,3.0,3.0,3.0,1.0,3.0,1.0,1.0,1.0,3.0,3.0,3.0,3.0,2.0,2.0,2.0,3.0,2.0,1.0,2.0,2.0,3.0,1.0,2.0,1.0,1.0,2.0,3.0,4.0,3.0,4.0,3.0,2.0,2.0,3.0,1.0,3.0,1.0,1.0,2.0,4.0,3.0,2.0,4.0,3.0,3.0,3.0,2.0,2.0,3.0,4.0,2.0,4.0,1.0,3.0,1.0,3.0,3.0,2.0,2.0,2.0,2.0,3.0,2.0,3.0,3.0,3.0,3.0,2.0,2.0,2.0,1.0,1.0,2.0,3.0,2.0,1.0,2.0,1.0,1.0,72.0,1.0,8.445652,0.6

### Cleaning Data Types And Unique Values

In [492]:
# Drop columns with only one unique value
merged_df = merged_df.loc[:, merged_df.apply(pd.Series.nunique) != 1]

# Convert ID columns to int and categorical float columns to float
for col in merged_df.columns:
    if col.startswith('ID'):
        merged_df[col] = merged_df[col].astype(int)
    if merged_df[col].dtype == 'category' and merged_df[col].cat.categories.dtype == 'float64':
        merged_df[col] = merged_df[col].astype(float)
        
# print("\nCleaned Merged DataFrame:")
# print(merged_df.shape)
# print(merged_df.dtypes)
merged_df.head(10)

,IDBOOK,IDSCHOOL,IDCLASS,IDSTUD,ITSEX,BSDAGE,ITADMINI,ILRELIAB,ITLANG_SQ,LCID_SQ,BSBG01,BSBG03,BSBG04,BSBG05A,BSBG05B,BSBG05C,BSBG05D,BSBG05E,BSBG05F,BSBG05G,BSBG05H,BSBG05I,BSBG05J,BSBG06A,BSBG06B,BSBG07,BSBG08A,BSBG08B,BSBG09A,BSBG10,BSBG11A,BSBG11B,BSBG12A,BSBG12B,BSBG12C,BSBG12D,BSBG12E,BSBG12F,BSBG13A,BSBG13B,BSBG13C,BSBG13D,BSBG13E,BSBG13F,BSBG13G,BSBG14A,BSBG14B,BSBG14C,BSBG14D,BSBG14E,BSBG14F,BSBG14G,BSBG14H,BSBG14I,BSBG15A,BSBG15B,BSBG15C,BSBG15D,BSBG15E,BSBG15F,BSBG16A,BSBG16B,BSBG16C,BSBG16D,BSBG16E,BSBG16F,BSBG16G,BSBG16H,BSBG17A,BSBG17B,BSBG17C,BSBG17D,BSBG17E,BSBG17F,BSBG17G,BSBG17H,BSBG17I,BSBG17J,BSBG17K,BSBG17L,BSBG17M,BSBG17N,BSBM18,BSBM19A,BSBM19B,BSBM19C,BSBM19D,BSBM19E,BSBM19F,BSBM19G,BSBM19H,BSBM19I,BSBM20A,BSBM20B,BSBM20C,BSBM20D,BSBM20E,BSBM20F,BSBM20G,BSBM20H,BSBM20I,BSBM20J,BSBM21A,BSBM21B,BSBM21C,BSBM21D,BSBM21E,BSBM21F,BSBM22A,BSBM22B,BSBM22C,BSBM22D,BSBM22E,BSBM22F,BSBM22G,BSBM22H,BSBM23A,BSBM23B,BSBM23C,BSBM23D,BSBM23E,BSBM23F,BSBM23G,BSBM23H,BSBM23I,BSBM30A,BSBM49A,JKZONE,JKREP,TOTWGT,HOUWGT,SENWGT,WGTFAC1,WGTADJ1,WGTFAC2,WGTADJ3,BSMMAT01,BSMMAT02,BSMMAT03,BSMMAT04,BSMMAT05,BSMNUM01,BSMNUM02,BSMNUM03,BSMNUM04,BSMNUM05,BSMALG01,BSMALG02,BSMALG03,BSMALG04,BSMALG05,BSMGEO01,BSMGEO02,BSMGEO03,BSMGEO04,BSMGEO05,BSMDAT01,BSMDAT02,BSMDAT03,BSMDAT04,BSMDAT05,BSMKNO01,BSMKNO02,BSMKNO03,BSMKNO04,BSMKNO05,BSMAPP01,BSMAPP02,BSMAPP03,BSMAPP04,BSMAPP05,BSMREA01,BSMREA02,BSMREA03,BSMREA04,BSMREA05,BSMIBM01,BSMIBM02,BSMIBM03,BSMIBM04,BSMIBM05,BSSIBM01,BSSIBM02,BSSIBM03,BSSIBM04,BSSIBM05,BSBGHER,BSDGHER,BSBGSSB,BSDGSSB,BSBGSB,BSDGSB,BSBGSLM,BSDGSLM,BSBGICM,BSDGICM,BSBGDML,BSDGDML,BSBGSCM,BSDGSCM,BSBGSVM,BSDGSVM,BSBGSLS,BSDGSLS,BSBGICS,BSDGICS,BSBGSCS,BSDGSCS,BSBGSVS,BSDGSVS,BSBGSEC,BSDGSEC,BSBGDSL,BSDGDSL,BSBGVEP,BSDGVEP,BSDG05S,BSDGEDUP,BSDMLOWP,BSDSLOWP,BSDMZERP,BSDSZERP,BSDMCORP,BSDSCORP,BSDMMISP,BSDSMISP,IDTEACH,IDLINK,IDTEALIN,IDSUBJ,ITCOURSE,MATSUBJ,SCISUBJ,NSTEACH,NTEACH,TCHWGT,MATWGT,SCIWGT
0,1,5001,500101,50010101,1.0,14.17,3.0,1.0,1.0,6153.0,1.0,3.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,8.0,8.0,6.0,2.0,2.0,1.0,5.0,3.0,3.0,1.0,1.0,3.0,3.0,2.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,3.0,3.0,3.0,3.0,3.0,1.0,3.0,1.0,1.0,1.0,3.0,3.0,3.0,3.0,2.0,2.0,2.0,3.0,2.0,1.0,2.0,2.0,3.0,1.0,2.0,1.0,1.0,2.0,3.0,4.0,3.0,4.0,3.0,2.0,2.0,3.0,1.0,3.0,1.0,1.0,2.0,4.0,3.0,2.0,4.0,3.0,3.0,3.0,2.0,2.0,3.0,4.0,2.0,4.0,1.0,3.0,1.0,3.0,3.0,2.0,2.0,2.0,2.0,3.0,2.0,3.0,3.0,3.0,3.0,2.0,2.0,2.0,1.0,1.0,2.0,3.0,2.0,1.0,2.0,1.0,1.0,72.0,1.0,8.445652,0.601726,0.059109,4.625,1.0,1.5,1.217391,579.28271,619.61052,629.98681,635.76196,622.21290,616.15050,631.59033,586.11926,638.12500,662.16888,578.31469,623.23130,628.23201,621.59125,637.92312,587.19405,575.88615,594.31813,592.97973,579.86785,612.05582,627.74179,602.25184,623.19325,604.42856,650.54364,605.49470,606.24526,666.36207,619.27620,590.75504,595.06220,592.73813,636.12174,610.08847,621.33201,626.64541,627.60488,646.77978,623.79722,4.0,4.0,5.0,5.0,4.0,5.0,5.0,5.0,5.0,5.0,11.16547,2.0,9.23501,2.0,7.83267,3.0,9.08471,3.0,7.71297,3.0,9.05919,2.0,9.72439,3.0,9.19823,2.0,12.68366,1.0,12.24370,1.0,10.19911,2.0,12.44879,1.0,12.74515,1.0,9.83635,2.0,8.43770,2.0,3.0,6.0,2.0,2.0,2.0,2.0,72.72727,72.72727,0.00000,0.00000,500102,2,50010202,1,1.0,1.0,0.0,1.0,2.0,4.222826,8.445652,14.087339
1,1,5001,500101,50010101,1.0,14.17,3.0,1.0,1.0,6153.0,1.0,3.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,8.0,8.0,6.0,2.0,2.0,1.0,5.0,3.0,3.0,1.0,1.0,3.0,3.0,2.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,3.0,3.0,3.0,3.0,3.0,1.0,3.0,1.0,1.0,1.0,3.0,3.0,3.0,3.0,2.0,2.0,2.0,3.0,2.0,1.0,2.0,2.0,3.0,1.0,2.0,1.0,1.0,2.0,3.0,4.0,3.0,4.0,3.0,2.0,2.0,3.0,1.0,3.0,1.0,1.0,2.0,4.0,3.0,2.0,4.0,3.0,3.0,3.0,2.0,2.0,3.0,4.0,2.0,4.0,1.0,3.0,1.0,3.0,3.0,2.0,2.0,2.0,2.0,3.0,2.0,3.0,3.0,3.0,3.0,2.0,2.0,2.0,1.0,1.0,2.0,3.0,2.0,1.0,2.0,1.0,1.0,72.0,1.0,8.445652,0.601726,0.059109,4.625,1.0,1.5,1.217391,579.28271,619.61052,629.98681,635.76196,622.21290,616.15050,631.59033,586.11926,638.12500,662.16888,578.31469,623.23130,628.23201,621.59125,

In [493]:
print("\nCleaned Merged DataFrame:")
print(merged_df.shape)
print(merged_df.dtypes)
print(merged_df.head(200))


Cleaned Merged DataFrame:
(9992, 238)
IDBOOK        int32
IDSCHOOL      int32
IDCLASS       int32
IDSTUD        int32
ITSEX       float64
             ...   
NSTEACH     float64
NTEACH      float64
TCHWGT      float64
MATWGT      float64
SCIWGT      float64
Length: 238, dtype: object
     IDBOOK  IDSCHOOL  IDCLASS    IDSTUD  ITSEX  BSDAGE  ITADMINI  ILRELIAB  \
0         1      5001   500101  50010101    1.0   14.17       3.0       1.0   
1         1      5001   500101  50010101    1.0   14.17       3.0       1.0   
2         9      5001   500101  50010102    1.0   14.58       3.0       1.0   
3         9      5001   500101  50010102    1.0   14.58       3.0       1.0   
4         2      5001   500101  50010103    1.0   13.75       3.0       1.0   
..      ...       ...      ...       ...    ...     ...       ...       ...   
195      11      5005   500502  50050216    1.0   14.50       3.0       1.0   
196      11      5005   500502  50050216    1.0   14.50       3.0       1.0   
197

In [494]:
import os, sys

ROOT = "./data"
if not os.path.exists(ROOT):
    os.makedirs(ROOT)

merged_df.reset_index(drop=True, inplace=True)
merged_df.to_feather(f"{ROOT}/cleaned_data.feather")

print("DataFrame saved to './data/cleaned_data.feather'")

DataFrame saved to './data/cleaned_data.feather'
